<a href="https://colab.research.google.com/github/Sashank11/learn_tensorflow/blob/main/Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP has the goal of deriving info out of natural language(sequences)

Another common term for NLP problems is seq2seq.

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fd326254-84e3-07dd-8bd0-f6de1fc7b2cc)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2023-03-08 06:49:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-08 06:49:00 (76.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

intro to NLP dataset text sample of tweets labelled as disaster/not disaster

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-03-08 06:49:03--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   738KB/s    in 0.8s    

2023-03-08 06:49:04 (738 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Unzip data
unzip_data("nlp_getting_started.zip")

# Visualize

In [6]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][3]

'13,000 people receive #wildfires evacuation orders in California '

In [8]:
#shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# looking at test dataframe
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# no in each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
#how many total samples
len(train_df), len(test_df)

(7613, 3263)

In [12]:
# visualize random training samples
import random
random_index = random.randint(0, len(train_df)- 5) # create random indexes not higher than the total number
for row in train_df_shuffled[["text", "target"]][random_index: random_index + 5].itertuples():
  _, text, target = row # remove stuff
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("----\n")


Target: 1 (real diaster)
Text: 
Driver rams car into Israeli soldiers wounds 3: military: A driver rammed a car into a group of Israeli soldi... http://t.co/oBSZ45ybAJ

----

Target: 0 (not real disaster)
Text: 
The advantages apropos of in flames favorable regard mississauga ontario: pWHvGwax

----

Target: 1 (real diaster)
Text: 
Migrants drown at sea after boat capsizes off #Libya http://t.co/t4pv0nrOoV http://t.co/PSeYLYzck4

----

Target: 0 (not real disaster)
Text: 
Why Are #BAYONETS Being Distributed To Local Police Departments?
@RandPaul wants to know

https://t.co/XB8nfxaBUM

#EvilEmpire
#JadeHelm15

----

Target: 0 (not real disaster)
Text: 
Dysfunctional McConnell plans on holding Judicial Nominations hostage. Another example of how GOP canÛªt govern. http://t.co/VT2akY5MgK Û_

----



### Split data to train and val data

In [13]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size =0.1,
                                                                            random_state = 42)

In [14]:
# check len
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
len(train_df_shuffled)

7613

In [45]:
# check first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Convert text to numbers

### Text Vectorization (tokenization)

In [42]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [41]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens = None, # how  many words in vocab
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None, # creat group of n-words
                                    output_mode = "int", # how to map tokens to numbers
                                    output_sequence_length = None, # padded to the longest seq if none
                                    #pad_to_max_tokens = True
                                    )


In [43]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split())for i in train_sentences])/len(train_sentences))

15

In [44]:
# Setup text vec vars
max_vocab_lengths = 10000 # max number of words to have in our vocab
max_length = 15 # max length our sequences will be

In [46]:
text_vectorizer = TextVectorization(max_tokens = max_vocab_lengths,
                                    output_mode = "int",
                                    output_sequence_length = max_length,
                                    pad_to_max_tokens = True)